In [1]:
import sys, os, time
import numpy as np
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from process_travis import *

In [3]:
from utility import *

In [4]:
# create travis obj
process_Travis = ProcessTravisCIBuilds("WMlSAymr_DTc5XHe5mCj4w")

In [5]:
# seglearn latest build list
build_list = process_Travis.get_build_details('notthatanonymous/delfi', 50)

In [6]:
def process_logs(build_list):
        '''
        :param build_list:
        :return:
        '''
        run = 0
        data_list = []
        for build_id, build_desc in build_list.items():

            run += 1
            status = build_desc['status']
            while status not in ['passed', 'errored', 'failed']:
                time.sleep(30)
                status = process_Travis.get_build_info(build_id)

            print(f"{build_id} completed with status {status}.")

            if status == 'passed':
                for job_id in build_list[build_id]['job_list']:
                    job_details = process_Travis.get_job_details(job_id)
                    try:
                        score = job_details['log'].split("\nScore: ")[1].split("\n")[0]
                    except:
                        score = np.nan
                    data_list.append(list(np.append(job_details['job_config'].split('_'),
                                                    [int(run), round(float(score), 6),
                                                     job_details['processing_time']])))
            else:
                run -= 1

        return data_list

In [7]:
import pandas as pd

df = pd.DataFrame(process_logs(build_list), columns=['OS', 'Python', 'Hardware', 'Run', 'Score', 'Processing_Time'])

264433960 completed with status passed.
264433941 completed with status passed.
264433927 completed with status passed.
264433909 completed with status passed.
264433892 completed with status passed.
264433880 completed with status passed.
264433869 completed with status passed.
264433861 completed with status passed.
264433843 completed with status passed.
264433834 completed with status passed.
264433817 completed with status passed.
264433799 completed with status passed.
264433788 completed with status passed.
264433775 completed with status passed.
264433756 completed with status passed.
264433747 completed with status passed.
264433731 completed with status passed.
264433722 completed with status passed.
264433714 completed with status passed.
264433703 completed with status passed.
264433691 completed with status passed.
264433680 completed with status passed.
264433670 completed with status passed.
264433658 completed with status passed.
264433642 completed with status passed.


In [8]:
df["Project"] = ['delfi'] * df.shape[0]

In [9]:
df.shape

(400, 7)

In [10]:
df.head()

,OS,Python,Hardware,Run,Score,Processing_Time,Project
0,Linux-Xenial,3.8,amd64,1.0,5.150375,1268.0,delfi
1,Linux-Xenial,3.6,amd64,1.0,5.150375,1658.0,delfi
2,Linux-Xenial,3.7,amd64,1.0,5.150375,1166.0,delfi
3,Linux-Bionic,3.7,amd64,1.0,5.150375,1150.0,delfi
4,Linux-Focal,3.7,amd64,1.0,5.209411,1261.0,delfi


In [11]:
write_df(df = df, file_name='delfi.csv')

In [16]:
# remove IBM CPU rows from rrcf
rrcf_df = pd.read_csv('../output/rrcf.csv')
rrcf_df.head(10)

,OS,Python,Hardware,Run,Score,Processing_Time,Project
0,Linux-Xenial,3.8,amd64,1.0,9.038623,57.0,rrcf
1,Linux-Xenial,3.6,amd64,1.0,8.600704,60.0,rrcf
2,Linux-Xenial,3.7,amd64,1.0,8.900525,53.0,rrcf
3,Linux-Bionic,3.7,amd64,1.0,8.552067,53.0,rrcf
4,Linux-Focal,3.7,amd64,1.0,8.455796,54.0,rrcf
5,Linux-Xenial,3.7,arm64,1.0,8.516388,89.0,rrcf
6,Linux-Xenial,3.7,s390x,1.0,7.299293,997.0,rrcf
7,MacOS,3.7,amd64,1.0,8.258037,117.0,rrcf
8,Windows,3.7,amd64,1.0,8.724733,120.0,rrcf
9,Linux-Xenial,3.8,amd64,2.0,8.764135,57.0,rrcf


In [18]:
rrcf_df = rrcf_df[rrcf_df.Hardware != 's390x']

In [19]:
rrcf_df.head(10)

,OS,Python,Hardware,Run,Score,Processing_Time,Project
0,Linux-Xenial,3.8,amd64,1.0,9.038623,57.0,rrcf
1,Linux-Xenial,3.6,amd64,1.0,8.600704,60.0,rrcf
2,Linux-Xenial,3.7,amd64,1.0,8.900525,53.0,rrcf
3,Linux-Bionic,3.7,amd64,1.0,8.552067,53.0,rrcf
4,Linux-Focal,3.7,amd64,1.0,8.455796,54.0,rrcf
5,Linux-Xenial,3.7,arm64,1.0,8.516388,89.0,rrcf
7,MacOS,3.7,amd64,1.0,8.258037,117.0,rrcf
8,Windows,3.7,amd64,1.0,8.724733,120.0,rrcf
9,Linux-Xenial,3.8,amd64,2.0,8.764135,57.0,rrcf
10,Linux-Xenial,3.6,amd64,2.0,8.841522,61.0,rrcf


In [20]:
write_df(df = rrcf_df, file_name='rrcf.csv')